# Viewing results in Napari

Code uses outputs created by the spot_detection script to quickly load a particular image with masks and spots into napari in order to visually evaluate the restults.

Run this on a local machine rather than the server as getting X11 forwarding to work on the server is a bit tricky. 

Pathnames from the fileList.csv df are automatically converted from /mnt/ to /Volumes/

In [ ]:
import napari
from skimage.measure import regionprops_table, regionprops
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import gc
from bioio import BioImage
import bioio_nd2
import bioio_tifffile
from bioio.writers import OmeTiffWriter


anisotropy = (3,1,1) # Relative scale of (Z,X,Y) axes

nucChannel = 0 # red emerin rings
spotChannel = 1 # green spots

# in lucien's original scripts:
# channel 0 is green spots
# channel 1 is red emerin

In [11]:
output_path = '/Volumes/external.data/MeisterLab/jsemple/lhinder/segmentation_Dario/DPY27/'

df = pd.read_csv(output_path + 'fileList.csv')
indices = range(0,len(df))
df

,filename,date,experiment,strain,protein,id,raw_filepath,denoised_filepath
0,20240915_1268_E_bean_15um,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_bean_15um,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
1,20240915_1268_E_bean_15um_02,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_bean_15um_02,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
2,20240915_1268_E_early_15um,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
3,20240915_1268_E_early_15um_02,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_02,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
4,20240915_1268_E_early_15um_03,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_03,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
5,20240915_1268_E_early_15um_04,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_04,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
6,20240915_1268_E_early_15um_05,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_05,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
7,20240915_1268_E_early_15um_06,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_06,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
8,20240915_1268_E_early_15um_07,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_07,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
9,20240915_1268_E_late_15um,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_late_15um,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...


In [20]:
index=indices[12] # choose one index at a time

#def view_spots_in_napari(df, index, anisotropy=(4,1,1)):
# Load the data
masks = BioImage(output_path+'segmentation/'+df.id.iloc[index]+'.tif', reader=bioio_tifffile.Reader)
masks = masks.get_image_data("ZYX", T=0, C=0)

img = BioImage(df.raw_filepath.iloc[index].replace('/mnt','/Volumes'), reader=bioio_nd2.Reader)
img = img.get_image_data("ZYXC", T=0)

df_features = pd.read_csv(output_path+'spots/'+df.id.iloc[index]+'.csv')





/Users/semple/micromamba/envs/lhcellpose/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


In [21]:

points = np.array(df_features[df_features['signal']>0][['z','y','x']].values)
points[:,0] = points[:,0]*anisotropy[0]
points[:,1] = points[:,1]*anisotropy[1]
points[:,2] = points[:,2]*anisotropy[2]

### Display everything in Napari
viewer = napari.Viewer()
layer_c0 = viewer.add_labels(masks ,scale=anisotropy,blending='additive',opacity=0.3)
layer_c0 = viewer.add_image(img[:,:,:,nucChannel] ,scale=anisotropy,blending='additive', colormap='magenta',opacity=0.85)
layer_c0 = viewer.add_image(img[:,:,:,spotChannel] ,scale=anisotropy,blending='additive',colormap='cyan')
points_layer = viewer.add_points(points, size=3,blending='additive',face_color='yellow')
# make sure blending is additive